# Data Analytics Project Work
### File for testing different stuff for the project :D

In [40]:
# imports
import pandas as pd
import numpy as np

# Testing on reading one of the datasets and creating a Pandas DataFrame - object
test_df = pd.read_csv("HybridDataset_for_RSSbasedLocalization-main/Scenario 0/RPI/RSS_BLT_Dataset/Distanza10.csv", sep=';')
test_df.head()

,Timestamp,Rx Power [dBm]
0,Fri Mar 24 14:13:33 2023,-79
1,Fri Mar 24 14:13:34 2023,-70
2,Fri Mar 24 14:13:36 2023,-73
3,Fri Mar 24 14:13:38 2023,-79
4,Fri Mar 24 14:13:39 2023,-76


In [41]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Timestamp       517 non-null    object
 1   Rx Power [dBm]  517 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ KB


Everything seemed to be working great as we got the first five entries of the dataset with a .head()– method and some information about the dataset with the .info()– method.

In [42]:
# In this cell a we create a Dataframe-object for each csv-file in the Scenario A -> RPI -> BLT measurement.
# Then we add all the dataframes into an anchor name-dataframe dict so we can access them later on.
# This cell can be altered to a function to make it more general and usable with the other measurement cases also.

anchors = ["Anchor 1.csv", "Anchor 2.csv", "Anchor 3.csv", "Anchor 4.csv", "Anchor 5.csv", "Anchor 6.csv"]
scenarioa_blt_dataframes = {}
directory_path = "HybridDataset_for_RSSbasedLocalization-main/Scenario A/RPI/RSS_BLT_Dataset/"

for anchor in anchors:
    current_df = pd.read_csv(directory_path + anchor, sep=';')
    scenarioa_blt_dataframes[anchor.replace(".csv","")] = current_df

print(scenarioa_blt_dataframes["Anchor 1"])

                    Timestamp  Rx Power [dBm] Relative Coordinates [m]  \
0    Wed Jan 25 15:20:33 2023             -77                 5.55, 15   
1    Wed Jan 25 15:20:35 2023             -78                 5.55, 15   
2    Wed Jan 25 15:20:36 2023             -68                 5.55, 15   
3    Wed Jan 25 15:20:37 2023             -70                 5.55, 15   
4    Wed Jan 25 15:20:38 2023             -77                 5.55, 15   
..                        ...             ...                      ...   
564  Wed Jan 25 15:35:25 2023             -71                 5.55, 15   
565  Wed Jan 25 15:35:26 2023             -74                 5.55, 15   
566  Wed Jan 25 15:35:30 2023             -71                 5.55, 15   
567  Wed Jan 25 15:35:31 2023             -70                 5.55, 15   
568  Wed Jan 25 15:35:32 2023             -71                 5.55, 15   

    Target Coordinates [m]  Distance Target - Anchor [m]  
0                     0, 0                        15

In [43]:
# This cell calculates the mean values over all the samples for each dataset and then adds them to an anchor name-mean value dict.
# This can also be altered to function for more general use.
# And also here we should calculate the mean over some amount of bursts instead of all samples.

mean_values = {}

for anchor in scenarioa_blt_dataframes:
    power_values = scenarioa_blt_dataframes[anchor]["Rx Power [dBm]"].to_list()
    average_value = np.mean(power_values)
    mean_values[anchor] = (average_value)

print(mean_values)

{'Anchor 1': -71.31985940246045, 'Anchor 2': -75.96064400715564, 'Anchor 3': -69.59061488673139, 'Anchor 4': -70.6858407079646, 'Anchor 5': -72.95810564663023, 'Anchor 6': -59.897271268057786}


In [44]:
# This cell calculates the estimated distances for each anchor using some given initial values from the article.
# As can be seen, the estimates are clearly wrong as the estimated distances are less than 1m. 
# This is why we should calculate the mean values over bursts and not over all the samples.

p_initial = -40
n_value = 4
estimated_distances = {}

for anchor in mean_values:
    estimated_d = 10**((mean_values[anchor]-p_initial)/(10*n_value))
    estimated_distances[anchor] = estimated_d

print(estimated_distances)

{'Anchor 1': 0.1648175730891101, 'Anchor 2': 0.12617807569471215, 'Anchor 3': 0.1820684218601984, 'Anchor 4': 0.17094404734994115, 'Anchor 5': 0.14998483816700325, 'Anchor 6': 0.31810333208308555}
